In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import tkinter as tk
from tkinter import *
import json
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\slope\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\slope\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:

# Unir los DataFrames en base al 'producto_id'
df_combined = pd.read_csv('productos_prueba.csv')
df_combined

C:\Users\slope\AppData\Local\Temp\ipykernel_15152\2315272005.py:2: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_combined = pd.read_csv('productos_prueba.csv')


,precio,producto_id,sucursal_id,id,marca,nombre,presentacion
0,29.90,0000000002288,2-1-009,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
1,32.90,0000000002288,2-1-037,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
2,36.90,0000000002288,2-1-090,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
3,39.90,0000000002288,2-3-247,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
4,499.99,0000000205870,9-1-430,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg
...,...,...,...,...,...,...,...
413140,149.99,9569753142128,25-1-1,9569753142128,DELI-SITAS,Milhojas Cobertura de Chocolate Blanco Deli-Si...,500.0 gr
413141,34.99,9795403001143,25-1-1,9795403001143,MAYO,Mini Pizzetas Mayo 12 Un,12.0 un
413142,312.50,9990385651922,5-1-4,9990385651922,DANA,Te Negro en Hebras Lata Dana 50 Gr,50.0 gr
413143,312.50,9990385651939,5-1-4,9990385651939,DANA,Te Verde en Hebras Lata Dana 50 Gr,50.0 gr


In [3]:
product_names = df_combined['nombre'].unique()
print(product_names.size)
product_names

53973


array(['Perejil Atado La Anonima 1 Un', 'Ojo de Bife 1 Kg',
       'Milanesa de Peceto Novillito 1 Kg', ...,
       'Te Negro en Hebras Lata Dana 50 Gr',
       'Te Verde en Hebras Lata Dana 50 Gr',
       'Yerba Mate Aromatizada Lata Dana 150 Gr'], dtype=object)

In [4]:
lemmatizer = WordNetLemmatizer()

# Datos de entrenamiento
intents = {
    "intents": [
        {"tag": "despedida",
         "patterns": ["Adiós", "Nos vemos", "Hasta luego", "Bye"],
         "responses": ["Adiós, que tengas un buen día", "Hasta luego, cuídate", "Nos vemos, ¡adiós!"]},
        {"tag": "gracias",
         "patterns": ["Gracias", "Muchas gracias", "Te lo agradezco"],
         "responses": ["De nada", "¡No hay problema!", "A la orden"]},
        {"tag": "nombre",
         "patterns": ["¿Cómo te llamas?", "¿Cuál es tu nombre?", "Dime tu nombre"],
         "responses": ["Soy un chatbot, tu asistente virtual", "Me llamo Chatbot", "Soy Chatbot, encantado de ayudarte"]},
        {"tag": "comida",
         "patterns": ["¿Cuál es tu comida favorita?", "¿Te gusta la comida?", "Dime algo sobre la comida"],
         "responses": ["No como, pero sé que hay muchas comidas deliciosas", "Los humanos disfrutan de una gran variedad de alimentos", "La comida es esencial para los humanos, pero yo no la necesito"]},
        {"tag": "greeting",
            "patterns": ["Hola", "Hey", "Hola, ¿cómo estás?", "Buenos días", "Buenas tardes"],
            "responses": ["Hola", "Hola, ¿cómo puedo ayudarte?", "Hola, ¿qué necesitas?"]},
        {"tag": "product_query",
            "patterns": ["¿Cuál es el precio de [producto]?", "Precio de [producto]", "¿Cuánto cuesta [producto]?"],
            "responses": ["Déjame buscar el precio para ti."]},
        {
            "tag": "products_list",
            "patterns": ["¿Qué productos tienes?", "Lista de productos", "¿Qué vendes?"],
            "responses": ["Estos son los productos disponibles: "]
        },

    ]
}

with open('intents.json', 'w') as outfile:
    json.dump(intents, outfile)

with open('intents.json') as file:
    data = json.load(file)

words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']



# Procesamiento de datos
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)

train_x = []
train_y = []

for i in training:
    train_x.append(i[0])
    train_y.append(i[1])

train_x = np.array(train_x)
train_y = np.array(train_y)


In [5]:
# Función para limpiar y lematizar oración
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Función para convertir oración a bolsa de palabras
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return(np.array(bag))

# Función para predecir la clase
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    print(return_list)
    return return_list

# Función para obtener respuesta
'''def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    print(list_of_intents)
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result'''

def get_response(ints, intents_json, msg):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    print(list_of_intents)


    if tag == 'products_list':
        response = "Estos son los productos disponibles:\n" + "\n".join(product_names)
        return response
    elif tag == 'product_query':
        product_name = msg.split('de')[-1].strip()
        prices = get_product_prices(product_name)
        if prices:
            response = f"Estos son los 10 precios más bajos que encontré para {product_name}:\n"
            sorted_prices = sorted(prices, key=lambda x: x['precio'])[:10]
            for price in sorted_prices:
                response += "- Sucursal {}: ${:.2f}\n".format(price['sucursal_id'], price['precio'])
            return response
        else:
            return "Lo siento, no encontré ese producto."
    else:
        for intent in data['intents']:
            if intent['tag'] == tag:
                responses = intent['responses']
        return random.choice(responses)    

def get_product_prices(product_name):
    result = df_combined[df_combined['nombre'].str.contains(product_name, case=False, na=False)]
    if not result.empty:
        return result[['sucursal_id', 'precio', 'marca', 'nombre', 'presentacion']].to_dict(orient='records')
    else:
        return None


In [6]:
class Chatbot:
    def __init__(self):
        self.model = self.create_model()
        self.train_model()
        self.intents = intents

    def create_model(self):
        model = Sequential()
        model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(len(train_y[0]), activation='softmax'))
        sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        return model

    def train_model(self):
        try:
            self.model.load_weights("model.weights.h5")
        except:
            self.model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
            self.model.save_weights("model.weights.h5")


    def chat(self, sentence):
        ints = predict_class(sentence, self.model)
        res = get_response(ints, self.intents, sentence)
        return res


chatbot = Chatbot()


Epoch 1/200


c:\Users\slope\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\slope\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1500 - loss: 2.0820      
Epoch 2/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 2.1044 
Epoch 3/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2028 - loss: 1.9216     
Epoch 4/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3056 - loss: 1.8592 
Epoch 5/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2944 - loss: 1.7696     
Epoch 6/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3306 - loss: 1.8425 
Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4306 - loss: 1.7317 
Epoch 8/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6417 - loss: 1.6307 
Epoch 9/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4750 - loss: 1.5291 
Epoch 10/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5694 - loss: 1.4732 
Epoch 11/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5083 - loss: 1.4108 
Epoch 12/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6722 

In [7]:
class ChatGUI:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("Chatbot")
        self.chat_frame = Frame(self.window)
        self.chat_frame.pack(padx=10, pady=10)
        self.text_widget = Text(self.chat_frame, width=50, height=20)
        self.text_widget.pack(side=LEFT, padx=10, pady=10)
        self.text_widget.config(state=DISABLED)
        self.scrollbar = Scrollbar(self.chat_frame)
        self.scrollbar.pack(side=RIGHT, fill=Y)
        self.text_widget.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_widget.yview)
        self.entry = Entry(self.window, width=50)
        self.entry.pack(padx=10, pady=10)
        self.entry.bind("<Return>", self.send_message)
        self.send_button = Button(self.window, text="Enviar", command=self.send_message)
        self.send_button.pack(pady=10)

    def send_message(self, event=None):
        message = self.entry.get()
        self.entry.delete(0, END)
        if message:
            self.display_message(f"Tú: {message}\n")
            response = chatbot.chat(message)
            self.display_message(f"Bot: {response}\n")

    def display_message(self, message):
        self.text_widget.config(state=NORMAL)
        self.text_widget.insert(END, message)
        self.text_widget.config(state=DISABLED)
        self.text_widget.yview(END)

    def run(self):
        self.window.mainloop()

chat_gui = ChatGUI()
chat_gui.run()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[{'intent': 'product_query', 'probability': '0.973795'}]
[{'tag': 'despedida', 'patterns': ['Adiós', 'Nos vemos', 'Hasta luego', 'Bye'], 'responses': ['Adiós, que tengas un buen día', 'Hasta luego, cuídate', 'Nos vemos, ¡adiós!']}, {'tag': 'gracias', 'patterns': ['Gracias', 'Muchas gracias', 'Te lo agradezco'], 'responses': ['De nada', '¡No hay problema!', 'A la orden']}, {'tag': 'nombre', 'patterns': ['¿Cómo te llamas?', '¿Cuál es tu nombre?', 'Dime tu nombre'], 'responses': ['Soy un chatbot, tu asistente virtual', 'Me llamo Chatbot', 'Soy Chatbot, encantado de ayudarte']}, {'tag': 'comida', 'patterns': ['¿Cuál es tu comida favorita?', '¿Te gusta la comida?', 'Dime algo sobre la comida'], 'responses': ['No como, pero sé que hay muchas comidas deliciosas', 'Los humanos disfrutan de una gran variedad de alimentos', 'La comida es esencial para los humanos, pero yo no la necesito']}, {'tag': 'greeting', 'patterns': ['Hola', 'Hey', 'Hola, ¿cómo estás?'